In [2]:
import os
import numpy as np
import cv2
import threading
import pickle
import matplotlib.pyplot as plt


In [ ]:
import os
import cv2
import pickle
import numpy as np
import matplotlib.pyplot as plt
import subprocess
import json
import folium
import re
from tqdm import tqdm

global df


def from_dataframe_to_gps(video_path):
    """
    Return (latitude, longitude) for a video, using the parsed coordinates DataFrame.
    Falls back to None if the video number isn't found.
    """
    # Extract video number from filename, e.g. "video12.mp4" or "Vídeo_12.mov"
    basename = os.path.basename(video_path)
    match = re.search(r'(\d+)', basename)
    if not match:
        print(f"⚠️  Could not extract video number from {basename}")
        return None

    video_num = int(match.group(1))

    # Find matching row in DataFrame
    row = df.loc[df['video'] == video_num]
    if row.empty:
        print(f"⚠️  No coordinates found for video {video_num}")
        return None

    lat = float(row.iloc[0]['latitude'])
    lon = float(row.iloc[0]['longitude'])
    return lat, lon

def get_gps_from_video(video_path):
    """Extract single GPS coordinate (latitude, longitude) from video metadata."""
    cmd = [
        "ffprobe",
        "-v", "quiet",
        "-print_format", "json",
        "-show_entries", "format_tags",
        "-show_entries", "stream_tags",
        video_path
    ]
    result = subprocess.run(cmd, capture_output=True, text=True)
    print(video_path)
    metadata = json.loads(result.stdout or "{}")

    # Search for location info in both format and stream tags
    tags_candidates = []
    format_tags = metadata.get("format", {}).get("tags", {})
    if format_tags:
        tags_candidates.append(format_tags)

    streams = metadata.get("streams", [])
    for s in streams:
        if "tags" in s:
            tags_candidates.append(s["tags"])

    gps_raw = None
    for tags in tags_candidates:
        for key in ["com.apple.quicktime.location.ISO6709", "location", "location-eng"]:
            if key in tags:
                gps_raw = tags[key]
                break
        if gps_raw:
            break

    if not gps_raw:
        return None

    # Normalize and extract coordinates
    match = re.match(r"([+-]\d+\.\d+)([+-]\d+\.\d+)", gps_raw)
    if match:
        lat = float(match.group(1))
        lon = float(match.group(2))
        return lat, lon

    return None

def overlay_masks_on_image(image, masks, labels, alpha=0.4, show_labels=True):
    """Overlay segmentation masks on an image with random colors and centered labels."""
    overlay = image.copy()
    color_map = {}

    # Assign each label a random color (consistent seed)
    rng = np.random.default_rng(42)
    for label in np.unique(labels):
        color_map[label] = rng.integers(0, 255, size=3, dtype=np.uint8)

    # Auto-adjust font scale to image size
    h, w = image.shape[:2]
    base_font_scale = max(0.6, min(1.5, (w * h) / (1920 * 1080) * 1.0))
    font_thickness = int(max(1, base_font_scale * 2))

    for mask, label in zip(masks, labels):
        color = color_map[label]
        mask = mask.astype(bool)
        if not np.any(mask):
            continue

        # Blend per channel
        for c in range(3):
            overlay[:, :, c][mask] = (
                (1 - alpha) * overlay[:, :, c][mask] + alpha * color[c]
            ).astype(np.uint8)

        # --- Compute centroid for label placement ---
        if show_labels:
            M = cv2.moments(mask.astype(np.uint8))
            if M["m00"] > 0:
                cx = int(M["m10"] / M["m00"])
                cy = int(M["m01"] / M["m00"])

                # Draw text with outline
                cv2.putText(
                    overlay,
                    label,
                    (cx - 20, cy),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    base_font_scale,
                    (255, 255, 255),
                    font_thickness + 1,
                    cv2.LINE_AA,
                )
                cv2.putText(
                    overlay,
                    label,
                    (cx - 20, cy),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    base_font_scale,
                    (0, 0, 0),
                    font_thickness,
                    cv2.LINE_AA,
                )

    # --- Create legend on the right ---
    legend = np.ones((overlay.shape[0], 200, 3), dtype=np.uint8) * 255
    y = 30
    for label, color in color_map.items():
        cv2.rectangle(legend, (10, y - 15), (40, y + 5), color.tolist(), -1)
        cv2.putText(legend, label, (50, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)
        y += 25

    combined = np.hstack((overlay, legend))
    return combined



def hist_per_video(video_path, labels_of_interest):
    """
    Compute per-label area from SAM masks and generate overlays + histogram for all frames.
    Also saves a CSV with per-class percentage area coverage.
    """
    orig_dir = os.path.join(video_path, "original")
    lang_sam_dir = os.path.join(video_path, "lang_sam")

    if not (os.path.isdir(orig_dir) and os.path.isdir(lang_sam_dir)):
        print(f"⚠️ Missing 'original/' or 'lang_sam/' in {video_path}")
        return 0

    pkl_files = sorted([f for f in os.listdir(lang_sam_dir) if f.endswith(".pkl")])
    if not pkl_files:
        print(f"⚠️ No pickle files in {lang_sam_dir}")
        return 0

    area_dict = {}
    total_area = 0

    # --- Process each frame ---
    for idx, pkl_name in enumerate(pkl_files):
        pkl_path = os.path.join(lang_sam_dir, pkl_name)

        with open(pkl_path, "rb") as f:
            output = pickle.load(f)

        # Skip frames without valid segmentation
        if not isinstance(output[0].get("masks"), np.ndarray):
            print(f"⚠️ No masks found in {pkl_name}, skipping.")
            continue

        masks = output[0]["masks"].astype(bool)
        labels = output[0]["labels"]

        # --- Find corresponding image ---
        frame_num = re.findall(r"\d+", pkl_name)
        frame_num = frame_num[-1] if frame_num else f"{idx:05d}"
        img_candidates = sorted(
            [f for f in os.listdir(orig_dir) if frame_num in f and f.lower().endswith((".jpg", ".png"))]
        )
        if not img_candidates:
            all_imgs = sorted([f for f in os.listdir(orig_dir) if f.lower().endswith((".jpg", ".png"))])
            if idx < len(all_imgs):
                img_candidates = [all_imgs[idx]]
            else:
                print(f"⚠️ Could not match frame for {pkl_name}")
                continue

        img_path = os.path.join(orig_dir, img_candidates[0])
        image = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)

        # --- Overlay visualization ---
        #overlay = overlay_masks_on_image(image, masks, labels)
        #overlay_filename = f"seg_overlay_{frame_num}.png"
        #overlay_path = os.path.join(video_path, "segmentation_viz", overlay_filename)
        #print(overlay_path)
        #os.makedirs(os.path.dirname(overlay_path), exist_ok=True)
        #cv2.imwrite(overlay_path, cv2.cvtColor(overlay, cv2.COLOR_RGB2BGR))
        #print(f"🖼️ Saved overlay {overlay_filename}")

        # --- Compute per-label area ---
        for label, mask in zip(labels, masks):
            area = np.sum(mask)
            area_dict[label] = area_dict.get(label, 0) + area
            total_area += area

    # --- Compute relative (percentage) areas ---
    if total_area == 0:
        print(f"⚠️ No valid mask pixels in {video_path}")
        return 0

    area_percent = {label: (area / total_area * 100) for label, area in area_dict.items()}

    # --- Plot histogram (percentages) ---
    plt.figure()
    plt.bar(area_percent.keys(), area_percent.values(), color='skyblue')
    plt.ylabel("Percentage of segmented area (%)")
    plt.title(f"Area percentage per label for {os.path.basename(video_path)}")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()

    hist_path = os.path.join(video_path, "area_hist.png")
    plt.savefig(hist_path)
    plt.close()
    print(f"📊 Saved histogram to {hist_path}")

    # --- Save CSV with percentages ---
    import csv
    csv_path = os.path.join(video_path,"..", "segmentation_stats.csv")
    all_labels =  ["grass", "trees", "bus", "building","people","car","road"]
    values = [f"{area_percent[l]:.3f}" for l in labels]
    file_exists = os.path.exists(csv_path)

    with open(csv_path, "a+", newline="") as f:
        f.seek(0)
        existing_rows = [row[0] for row in csv.reader(f)]
        writer = csv.writer(f)
        # Write header only once
        if not file_exists:
            writer.writerow(["video"] + all_labels)
        # write the values in order of the corredsponding labels
        if os.path.basename(video_path) not in existing_rows:
            #writer.writerow([os.path.basename(video_path)] + values)
            writer.writerow([os.path.basename(video_path)] + [f"{area_percent.get(label, 0):.3f}" for label in all_labels])

    print(f"💾 Created and saved histogram data to {csv_path}")

    # --- Return total percentage for labels of interest ---
    return sum(area_percent.get(label, 0) for label in labels_of_interest)


def map_videos(video_folders, labels_of_interest=["trees", "grass"],out_filename="map.html", ball_clr="green"):
    """Plot circle markers for each video with radius ranked by tree area."""
    m = folium.Map(location=[38.736, -9.14], zoom_start=12)

    # Step 1: Gather data
    video_data = []
    for video_name in tqdm(video_folders):
        video_basename = os.path.basename(video_name)
        group_dir = os.path.dirname(os.path.dirname(video_name))  # e.g., g1/output/topo_alameda → g1/
        video_path = os.path.join(group_dir, f"{video_basename}.MOV")

        # fallback: try .mp4 if .MOV not found
        if not os.path.exists(video_path):
            video_path = os.path.join(group_dir, f"{video_basename}.mp4")

        gps = get_gps_from_video(video_path)
        area = hist_per_video(video_name, labels_of_interest)

        if video_name == "topo_alameda":
            gps = (38.737340, -9.129204)
        elif gps is None:
            print(f"No GPS found for {video_name}")
            continue

        # --- Save CSV with percentages ---
        import csv
        csv_path = os.path.join(video_name,"..", "gps.csv")
        file_exists = os.path.exists(csv_path)

        with open(csv_path, "a+", newline="") as f:
            f.seek(0)
            existing_rows = [row[0] for row in csv.reader(f)]
            writer = csv.writer(f)
            # Write header only once
            if not file_exists:
                writer.writerow(["video"] + ["lat"] + ["lon"])
            # Write only if the video name is not already in the file
            if os.path.basename(video_name) not in existing_rows:
                writer.writerow([os.path.basename(video_name)] + [gps[0]] + [gps[1]])

        print(f"💾 Created and saved gps data to {csv_path}")

        video_data.append((video_name, gps, area))

    if not video_data:
        print("⚠️ No valid videos found.")
        return

    # Step 2: Rank / normalize areas
    areas = np.array([a for _, _, a in video_data])
    min_area, max_area = np.min(areas), np.max(areas)
    if max_area == min_area:
        normalized = np.ones_like(areas)
    else:
        normalized = (areas - min_area) / (max_area - min_area)

    # Step 3: Map normalized areas to a visual radius range
    min_radius, max_radius = 10, 40
    radii = min_radius + normalized * (max_radius - min_radius)

    # Step 4: Plot circles with ranked sizes
    for (video_name, (lat, lon), area), radius in zip(video_data, radii):
        folium.CircleMarker(
            location=[lat, lon],
            radius=radius,
            color=ball_clr,
            fill=True,
            fill_opacity=0.6,
            popup=f"{video_name}: {int(area)} px² trees",
        ).add_to(m)

    m.save(out_filename)
    print(f"✅ Saved map to {out_filename}")



In [ ]:
base_dir = "."
#groups = [d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d)) and d.startswith("g")]
groups = ["g5"]

for group in groups:
    output_root = os.path.join(base_dir, group)
    if not os.path.isdir(output_root):
        continue

    # Collect all per-video output folders inside each group's output/
    video_folders = [
        os.path.join(output_root, d)
        for d in os.listdir(output_root)
        if os.path.isdir(os.path.join(output_root, d))
    ]

    if not video_folders:
        print(f"⚠️ No video outputs found in {group}/output")
        continue

    print(f"\n📂 Processing {group}: {len(video_folders)} videos")

    # Run vegetation map
    #map_videos(
    #    video_folders,
    #    labels_of_interest=["trees", "grass"],
    #    out_filename=os.path.join(output_root, f"{group}_green_map.html"),
    #    ball_clr="green",
    #)

    # Run urban map
    map_videos(
        video_folders,
        labels_of_interest=["road", "car", "bus", "building"],
        out_filename=os.path.join(output_root, f"{group}_urban_map.html"),
        ball_clr="grey",
    )


In [ ]:
os.listdir("./g1")

In [ ]:
%pip install k3d

In [ ]:
import os
import pickle
import numpy as np
import k3d

def visualize_vggt_pointcloud(video_path, downsample=10, conf_thresh=0.5):
    """
    Visualize VGGT 3D point cloud for a given video output folder.

    Args:
        video_path (str): e.g. './g1/output/topo_alameda'
        downsample (int): keep 1 every N points
        conf_thresh (float): confidence threshold on world_points_conf
    """
    vggt_dir = os.path.join(video_path, "vggt")
    if not os.path.isdir(vggt_dir):
        print(f"⚠️ No VGGT folder in {video_path}")
        return None

    # Load the latest (or largest) pickle file
    pkl_files = sorted(
        [f for f in os.listdir(vggt_dir) if f.endswith(".pkl")],
        key=lambda x: os.path.getsize(os.path.join(vggt_dir, x)),
        reverse=True
    )
    if not pkl_files:
        print(f"⚠️ No VGGT pickle found in {vggt_dir}")
        return None

    latest_pkl = os.path.join(vggt_dir, pkl_files[0])
    print(f"📦 Loading VGGT data from {latest_pkl}")

    with open(latest_pkl, "rb") as f:
        vggt_data = pickle.load(f)

    # --- Extract fields safely ---
    world_points = np.array(vggt_data.get("world_points") or vggt_data.get("wrld_points"))
    if world_points is None or world_points.size == 0:
        print(f"⚠️ No 'world_points' field found in {latest_pkl}")
        return None

    conf = np.array(vggt_data.get("world_points_conf", np.ones(world_points.shape[:-1])))
    conf = np.squeeze(conf)

    # --- Apply confidence mask ---
    mask_conf = conf > conf_thresh
    world_points = np.reshape(world_points, (-1, 3))[mask_conf.flatten(), :]

    # --- Handle RGB ---
    images = vggt_data.get("images")
    if images is not None:
        # images: (N, C, H, W) → (H*W*N, 3)
        rgb = np.transpose(images, (0, 2, 3, 1)).reshape(-1, 3)
        rgb = np.clip(rgb * 255.0, 0, 255).astype(np.uint8)[mask_conf.flatten(), :]
    else:
        rgb = np.full((world_points.shape[0], 3), 200, dtype=np.uint8)  # gray fallback

    # --- Downsample for speed ---
    if downsample > 1:
        world_points = world_points[::downsample]
        rgb = rgb[::downsample]

    # --- Convert to integer RGB for K3D ---
    rgb_int = (
        (rgb[:, 0].astype(np.uint32) << 16) +
        (rgb[:, 1].astype(np.uint32) << 8) +
        rgb[:, 2].astype(np.uint32)
    )

    # --- Visualize in K3D ---
    plot = k3d.plot(height=600)
    point_cloud = k3d.points(
        positions=world_points,
        colors=rgb_int,
        point_size=0.005,
        shader='3d'
    )
    plot += point_cloud
    plot.display()

    print(f"✅ Displayed {len(world_points):,} points from {os.path.basename(video_path)}")
    return plot


# Example usage:
video = "./g1/output/almirante_reis"
plot = visualize_vggt_pointcloud(video, downsample=10, conf_thresh=0.5)


In [67]:
import os
import json
import pandas as pd
from collections import defaultdict

def count_objects_in_video(video_yolo_json_dir, conf_threshold=0.25):
    """
    Count unique detected objects per class in a YOLO JSON folder,
    considering only detections above a confidence threshold.
    """
    counts = defaultdict(set)  # {class_name: set(track_ids)}
    json_files = sorted([f for f in os.listdir(video_yolo_json_dir) if f.endswith(".json")])

    for jf in json_files:
        jf_path = os.path.join(video_yolo_json_dir, jf)
        try:
            with open(jf_path, "r") as f:
                data = json.load(f)
        except Exception as e:
            print(f"⚠️ Failed to read {jf_path}: {e}")
            continue

        if not data or not isinstance(data, list):
            continue

        detections = data[0] if isinstance(data[0], list) else data
        for det in detections:
            conf = det.get("confidence", 0.0)
            if conf < conf_threshold:
                continue

            cls = det.get("class_name", "unknown")
            tid = det.get("track_id", None)

            # Handle missing or invalid tracking IDs
            if tid is None or tid == -1:
                tid = f"{jf}_{cls}"  # fallback unique id per frame

            counts[cls].add(tid)

    return {cls: len(tids) for cls, tids in counts.items()}

def process_group(group_dir, conf_threshold=0.25):
    """
    Process all videos inside a group/output/ folder and save a CSV there.
    """
    output_dir = os.path.join(group_dir)
    if not os.path.isdir(output_dir):
        print(f"⚠️ No output folder in {group_dir}")
        return

    video_stats = {}
    for video_name in os.listdir(output_dir):
        video_dir = os.path.join(output_dir, video_name)
        yolo_json_dir = os.path.join(video_dir, "yolo_json")
        if not os.path.isdir(yolo_json_dir):
            continue

        print(f"📦 Processing {group_dir}/{video_name} ...")
        counts = count_objects_in_video(yolo_json_dir, conf_threshold)
        if counts:
            video_stats[video_name] = counts

    if not video_stats:
        print(f"⚠️ No valid YOLO data found in {group_dir}")
        return

    # Normalize into DataFrame
    df = pd.DataFrame.from_dict(video_stats, orient="index").fillna(0).astype(int)
    df.index.name = "video_id"
    df.sort_index(inplace=True)
    df["total_objects"] = df.sum(axis=1)

    csv_path = os.path.join(output_dir, f"yolo_counts.csv")
    df.to_csv(csv_path)
    print(f"✅ Saved {csv_path}")
    return df

def merge_all_groups(base_dir=".", conf_threshold=0.25):
    """
    Loop over all group folders (g1, g2, g5-nogps, etc.) and create one CSV per group.
    """
    for group in sorted(os.listdir(base_dir)):
        group_path = os.path.join(base_dir, group)
        if not os.path.isdir(group_path) or not group.startswith("g"):
            continue
        process_group(group_path, conf_threshold)


#if __name__ == "__main__":
#    merge_all_groups(".", conf_threshold=0.65)

process_group("./g5", 0.7)

📦 Processing ./g5/IMG_8619 ...
📦 Processing ./g5/IMG_8599 ...
📦 Processing ./g5/IMG_8589 ...
📦 Processing ./g5/IMG_8684 ...
📦 Processing ./g5/IMG_8597 ...
📦 Processing ./g5/IMG_8607 ...
📦 Processing ./g5/IMG_8676 ...
📦 Processing ./g5/IMG_8643 ...
📦 Processing ./g5/IMG_8638 ...
📦 Processing ./g5/IMG_8688 ...
📦 Processing ./g5/IMG_8642 ...
📦 Processing ./g5/IMG_8672 ...
📦 Processing ./g5/IMG_8671 ...
📦 Processing ./g5/IMG_8610 ...
📦 Processing ./g5/IMG_8641 ...
📦 Processing ./g5/IMG_8646 ...
📦 Processing ./g5/IMG_8581 ...
📦 Processing ./g5/IMG_8658 ...
📦 Processing ./g5/IMG_8679 ...
📦 Processing ./g5/IMG_8686 ...
📦 Processing ./g5/IMG_8588 ...
📦 Processing ./g5/IMG_8630 ...
📦 Processing ./g5/IMG_8680 ...
📦 Processing ./g5/IMG_8625 ...
📦 Processing ./g5/IMG_8670 ...
📦 Processing ./g5/IMG_8666 ...
📦 Processing ./g5/IMG_8629 ...
📦 Processing ./g5/IMG_8586 ...
📦 Processing ./g5/IMG_8627 ...
📦 Processing ./g5/IMG_8675 ...
📦 Processing ./g5/IMG_8674 ...
📦 Processing ./g5/IMG_8635 ...
📦 Proces

,car,traffic light,person,bench,stop sign,bicycle,truck,bus,motorcycle,fire hydrant,chair,parking meter,refrigerator,total_objects
video_id,,,,,,,,,,,,,,
IMG_8581,1,0,12,0,0,0,0,1,0,0,0,0,0,14
IMG_8583,1,0,1,0,0,1,0,0,0,0,0,0,0,3
IMG_8584,7,0,0,0,0,0,0,0,0,0,0,0,0,7
IMG_8586,4,0,1,1,0,0,0,0,0,0,0,0,0,6
IMG_8588,0,0,3,0,0,0,0,0,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
IMG_8686,7,0,0,0,0,0,0,0,0,0,0,0,0,7
IMG_8688,1,0,0,0,0,0,0,0,0,0,0,0,0,1
IMG_8690,0,0,0,0,0,0,0,2,0,0,0,0,0,2


In [ ]:
import re
import pandas as pd

def dms_to_decimal(degrees, minutes, seconds, direction):
    """Convert DMS to decimal degrees."""
    decimal = float(degrees) + float(minutes)/60 + float(seconds)/3600
    if direction in ['S', 'W']:
        decimal *= -1
    return decimal

def parse_videos(text):
    pattern = re.compile(
        r"Vídeo\s*(\d+)\s*[–-]\s*"
        r"(\d+)°(\d+)’(\d+)”([NS])?\s*"
        r"(\d+)°(\d+)’(\d+)”([EW])?"
    )

    data = []
    for match in pattern.finditer(text):
        vid, lat_d, lat_m, lat_s, lat_dir, lon_d, lon_m, lon_s, lon_dir = match.groups()
        lat_dir = lat_dir or 'N'
        lon_dir = lon_dir or 'W'
        lat = dms_to_decimal(lat_d, lat_m, lat_s, lat_dir)
        lon = dms_to_decimal(lon_d, lon_m, lon_s, lon_dir)
        data.append({"video": int(vid), "latitude": lat, "longitude": lon})
    return pd.DataFrame(data)

# Example usage:
with open("coords.txt", "r", encoding="utf-8") as f:
    text = f.read()

df = parse_videos(text)
df.to_csv("video_coordinates.csv", index=False)
df

In [58]:
df

,video,latitude,longitude
0,1,38.716111,-9.142500
1,2,38.717778,-9.144167
2,3,38.717222,-9.143611
3,4,38.716944,-9.143056
4,5,38.716389,-9.142778
...,...,...,...
68,70,38.718611,-9.143611
69,71,38.716944,-9.142500
70,72,38.717500,-9.142778
71,73,38.716389,-9.142222
